Loading data

In [67]:
%%bash

mkdir data

wget -nv https://github.com/sebastian-lapuschkin/lrp_toolbox/blob/master/data/MNIST/train_images.npy?raw=true -O data/train_images.npy

wget -nv https://github.com/sebastian-lapuschkin/lrp_toolbox/blob/master/data/MNIST/train_labels.npy?raw=true -O data/train_labels.npy

2022-04-30 20:06:38 URL:https://raw.githubusercontent.com/sebastian-lapuschkin/lrp_toolbox/master/data/MNIST/train_images.npy [47040080/47040080] -> "data/train_images.npy" [1]
2022-04-30 20:06:39 URL:https://raw.githubusercontent.com/sebastian-lapuschkin/lrp_toolbox/master/data/MNIST/train_labels.npy [480080/480080] -> "data/train_labels.npy" [1]


In [2]:
import numpy as np
import mlp_utils
from mlp_utils import Dense, ReLU, LogitsSoftMaxCE, NNetwork
from tqdm import tqdm
from importlib import reload

reload(mlp_utils)

<module 'mlp_utils' from '/home/whoisjiji/Desktop/crafted_neural_networks/mlp_utils.py'>

In [3]:
X = np.load('data/train_images.npy').astype(float) / 255.
Y = np.load('data/train_labels.npy').astype('uint8').ravel()
nclasses = len(np.unique(Y))

In [4]:
def iterate_minibatches(inputs, labels, batchsize, shuffle=True):
    if shuffle:
        indices = np.random.permutation(len(labels))
    else:
        indices = np.arange(len(labels))
    for start_idx in range(0, len(labels), batchsize):
        batch_indices = indices[start_idx : start_idx + batchsize]
        
        yield inputs[batch_indices], labels[batch_indices]

In [7]:
# defining network

LR = 0.1
input_size = np.prod(X.shape[1:]) ## skipping batch dim
model = NNetwork(layers = [
    Dense(input_size, 2**7, LR),
    ReLU(),
    Dense(2**7, 2**5, LR),
    ReLU(),
    Dense(2**5,nclasses, LR),
],
    loss = LogitsSoftMaxCE()
)
BATCHSIZE = 32
rand_batch_indices = np.random.choice(len(X), BATCHSIZE)
print(f"Train accuracy before learning :: {(model.predict(X[rand_batch_indices]) == Y[rand_batch_indices]).mean()}")

Train accuracy before learning :: 0.0625


In [6]:
EPOCHS = 3

for epoch in tqdm(range(EPOCHS)):
    
    for x_batch, y_batch in iterate_minibatches(X, Y, BATCHSIZE):
        layer_inputs, logits = model.forward(x_batch) # FORWARD PASS
        
        # ENDING FORWARD (COMPUTING BATCH LOSS) + STARTING BACKWARD (COMPUTING LOGITS GRAD)
        batch_loss, grad_logits = model.compute_loss(logits, y_batch)
        
        model.backward(layer_inputs, grad_logits)  # BACKWARD (UPDATING weights)
    print(f'Train accuracy after {epoch + 1} epoch :: {(model.predict(X[rand_batch_indices]) == Y[rand_batch_indices]).mean()}')


 33%|████████████████████████████▎                                                        | 1/3 [00:06<00:13,  6.81s/it]

Train accuracy after 1 epoch :: 0.96875


 67%|████████████████████████████████████████████████████████▋                            | 2/3 [00:12<00:06,  6.35s/it]

Train accuracy after 2 epoch :: 1.0


100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:20<00:00,  6.73s/it]

Train accuracy after 3 epoch :: 1.0
